In [5]:
!pip install pyspark


     ---------------------------------------- 0.0/317.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.3 MB ? eta -:--:--
     -------------------------------------- 0.0/317.3 MB 162.5 kB/s eta 0:32:32
     -------------------------------------- 0.0/317.3 MB 187.9 kB/s eta 0:28:09
     -------------------------------------- 0.0/317.3 MB 217.9 kB/s eta 0:24:17
     -------------------------------------- 0.1/317.3 MB 403.5 kB/s eta 0:13:06
     ---------------------------------------- 0.4/317.3 MB 1.6 MB/s eta 0:03:14
     ---------------------------------------- 1.3/317.3 MB 4.4 MB/s eta 0:01:13
     ---------------------------------------- 2.2/317.3 MB 6.4 MB/s eta 0:00:50
     ---------------------------------------- 3.0/317.3 MB 7.6 MB/s eta 0:00:42
     ---------------------------------------- 3.4/317.3 MB 8.0 MB/s eta 0:00:40
     ---------------------------------------- 3.4/317.3 MB 8.0 MB/s eta 0:00:40
     ---------------------------------------- 3.4/317.

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

In [7]:
# Initialiser une session Spark avec le nom d'application "SalesDataAnalysis"
spark = SparkSession.builder.appName("SalesDataAnalysis").getOrCreate()


In [9]:
sales_df = spark.read.csv("supermarket_sales.csv", header=True, inferSchema=True)

In [10]:
sales_df = sales_df.withColumn("Date", to_date(col("Date"), "M/d/yyyy")).dropna()


In [11]:
sales_df.show(5)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|2019-01-05|2024-09-21 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22|2019-03-08|2024-09-21 10:29:00|       Cash|  76.4|            4.761904762| 

In [12]:
# Grouper les données par "Product line" (ligne de produits) et calculer la somme des ventes totales pour chaque ligne de produits
total_sales_by_product_line = sales_df.groupBy("Product line").sum("Total").orderBy("sum(Total)", ascending=False)
total_sales_by_product_line.show()


+--------------------+------------------+
|        Product line|        sum(Total)|
+--------------------+------------------+
|  Food and beverages|56144.844000000005|
|   Sports and travel|55122.826499999996|
|Electronic access...|54337.531500000005|
| Fashion accessories|         54305.895|
|  Home and lifestyle| 53861.91300000001|
|   Health and beauty|49193.739000000016|
+--------------------+------------------+



In [13]:
# Grouper les données par "City" (ville) et calculer la somme des ventes totales pour chaque ville
sales_by_city = sales_df.groupBy("City").sum("Total").orderBy("sum(Total)", ascending=False)
sales_by_city.show()


+---------+------------------+
|     City|        sum(Total)|
+---------+------------------+
|Naypyitaw|110568.70649999994|
|   Yangon| 106200.3705000001|
| Mandalay|106197.67199999996|
+---------+------------------+



In [14]:
# Grouper les données par "Payment" (méthode de paiement) et compter le nombre de transactions pour chaque méthode
transactions_by_payment_method = sales_df.groupBy("Payment").count().orderBy("count", ascending=False)
transactions_by_payment_method.show()


+-----------+-----+
|    Payment|count|
+-----------+-----+
|    Ewallet|  345|
|       Cash|  344|
|Credit card|  311|
+-----------+-----+



In [15]:
# Grouper les données par "Customer type" (type de client) et calculer la somme des ventes totales pour chaque type de client
sales_by_customer_type = sales_df.groupBy("Customer type").sum("Total").orderBy("sum(Total)", ascending=False)
sales_by_customer_type.show()


+-------------+------------------+
|Customer type|        sum(Total)|
+-------------+------------------+
|       Member|164223.44400000002|
|       Normal|158743.30500000005|
+-------------+------------------+



In [16]:
# Grouper les données par "Product line" (ligne de produits) et calculer la somme des quantités vendues pour chaque ligne de produits
top_selling_products = sales_df.groupBy("Product line").sum("Quantity").orderBy("sum(Quantity)", ascending=False)
top_selling_products.show()


+--------------------+-------------+
|        Product line|sum(Quantity)|
+--------------------+-------------+
|Electronic access...|          971|
|  Food and beverages|          952|
|   Sports and travel|          920|
|  Home and lifestyle|          911|
| Fashion accessories|          902|
|   Health and beauty|          854|
+--------------------+-------------+



In [17]:
# Grouper les données par mois (format "yyyy-MM") et calculer la somme des ventes totales pour chaque mois

from pyspark.sql.functions import date_format

sales_by_month = sales_df.groupBy(date_format("Date", "yyyy-MM").alias("Month")).sum("Total").orderBy("Month")
sales_by_month.show()


+-------+------------------+
|  Month|        sum(Total)|
+-------+------------------+
|2019-01|116291.86800000005|
|2019-02| 97219.37399999997|
|2019-03|109455.50700000004|
+-------+------------------+



In [18]:
# Grouper les données par "Product line" (ligne de produits) et calculer la note moyenne pour chaque ligne de produits
average_rating_by_product_line = sales_df.groupBy("Product line").avg("Rating").orderBy("avg(Rating)", ascending=False)
average_rating_by_product_line.show()


+--------------------+-----------------+
|        Product line|      avg(Rating)|
+--------------------+-----------------+
|  Food and beverages|7.113218390804598|
| Fashion accessories|7.029213483146067|
|   Health and beauty|7.003289473684212|
|Electronic access...| 6.92470588235294|
|   Sports and travel|6.916265060240964|
|  Home and lifestyle|           6.8375|
+--------------------+-----------------+



In [19]:
sales_by_gender = sales_df.groupBy("Gender").sum("Total").orderBy("sum(Total)", ascending=False)
sales_by_gender.show()


+------+------------------+
|Gender|        sum(Total)|
+------+------------------+
|Female|167882.92500000002|
|  Male|155083.82400000014|
+------+------------------+



In [20]:
from pyspark.sql.functions import dayofweek
# Grouper les données par jour de la semaine et calculer la somme des ventes totales pour chaque jour
sales_by_day_of_week = sales_df.groupBy(dayofweek("Date").alias("Day of Week")).sum("Total").orderBy("Day of Week")
sales_by_day_of_week.show()


+-----------+------------------+
|Day of Week|        sum(Total)|
+-----------+------------------+
|          1|44457.892499999994|
|          2| 37899.07799999999|
|          3| 51482.24550000001|
|          4|         43731.135|
|          5|45349.248000000014|
|          6| 43926.34050000002|
|          7| 56120.80949999999|
+-----------+------------------+



In [21]:
quantity_vs_price = sales_df.select("Unit price", "Quantity").groupBy("Unit price").avg("Quantity").orderBy("Unit price")
quantity_vs_price.show()


+----------+-------------+
|Unit price|avg(Quantity)|
+----------+-------------+
|     10.08|          7.0|
|     10.13|          7.0|
|     10.16|          5.0|
|     10.17|          1.0|
|     10.18|          8.0|
|     10.53|          5.0|
|     10.56|          8.0|
|     10.59|          3.0|
|     10.69|          5.0|
|     10.75|          8.0|
|     10.96|         10.0|
|     10.99|          5.0|
|     11.28|          9.0|
|     11.43|          6.0|
|     11.53|          7.0|
|     11.81|          5.0|
|     11.85|          8.0|
|     11.94|          3.0|
|     12.03|          2.0|
|     12.05|          5.0|
+----------+-------------+
only showing top 20 rows

